In [1]:
import os 
from sqlalchemy import create_engine
import pandas as pd

user1 = os.environ['PAYMEE_SONTATA_DB_USER']
password = os.environ['PAYMEE_SONTATA_DB_PASSWORD']
db = "Sonata_Connect"
server = "172.17.130.216"   

engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")


pd.set_option('display.max_columns', 500)
sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 6892;SET NOCOUNT OFF"""
df=pd.read_sql_query(sql_query,engine)

df = df.replace(['Prayagraj Division','Lucknow Division'],['Prayagraj', 'Lucknow'])
df = df.tail(-1)

userid_list = df['UserID'].unique().tolist()
print('useridlist', userid_list)

sql_query = f"""SET NOCOUNT ON;SELECT task_id, task_description, status, priority, assigned_to, created_by, completion_date, created_on, deadline_date, category, review, feedback FROM [{db}].[dbo].accounts_task_details ORDER BY created_on;SET NOCOUNT OFF"""
all_tasks_df=pd.read_sql_query(sql_query,engine)




useridlist [7540, 9739, 15183, 16278, 19113, 19115, 21350, 23659, 27066, 27068, 32060, 3557, 7535, 18094, 7587, 10855, 11856, 15190, 19125, 21360, 32147, 4783, 7541, 10849, 15184, 15187, 29643, 31069, 7662, 17560, 19109, 23725, 28117, 31995, 4366, 7546, 7555, 7563, 7577, 9741, 15186, 16326, 17756, 18039, 18086, 19126, 19128, 7556, 7575, 7665, 15185, 18097, 19111, 7560, 7640, 10851, 15189, 7663, 10846, 17945, 22622, 7567, 15111, 18060, 18090, 19127, 27105, 7569, 22544, 22621, 24803, 7114, 7465, 27188, 0, 18091, 31035, 17944, 18043, 18102, 29791, 18034, 25063, 1968, 31303, 7543, 18033, 30464, 31300, 31868, 1174, 30567, 30571, 31026, 1074, 15054, 4891, 14050, 19132, 21158, 31941, 7013, 24776, 27095, 27992, 30095, 16347, 30572, 30577, 18035, 27938, 31505, 18024, 30565, 31038, 2774, 7074, 10850, 18037, 18092, 31145, 24808, 31027, 1976, 7342, 17691, 17692, 30746, 4347, 7272, 7544, 31305, 16283, 21479, 30223, 31169, 31306, 32126, 23657, 18036, 4708, 27998, 7664, 18093, 18044, 18023, 27101, 31

In [2]:
div_df = df[(df['RoleId']==34)&(df['RoleName']=='DIVISION HEAD')][['U_BUID','UserID','UserName','buname']]
div_df.rename(columns={'U_BUID':'Div_BUID','buname':'DivName','UserID':'Div_UserID','UserName':'Div_UserName'}, inplace=True)
reg_df = df[(df['RoleId']==35)&(df['RoleName']=='REGION HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
reg_df.rename(columns={'U_BUID':'Region_BUID','ReportingBUId':'Region_ReportingBUId','buname':'RegionName','UserID':'Region_UserID','UserName':'Region_UserName'}, inplace=True)
hub_df = df[(df['RoleId']==36)&(df['RoleName']=='HUB HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
hub_df.rename(columns={'U_BUID':'Hub_BUID','ReportingBUId':'Hub_ReportingBUId','buname':'HubName','UserID':'Hub_UserID','UserName':'Hub_UserName'}, inplace=True)
bm_df = df[(df['RoleId']==13)&(df['RoleName']=='Branch Manager')][['U_BUID','buname','UserID','UserName','ReportingBUId']]
bm_df.rename(columns={'U_BUID':'BM_BUID','ReportingBUId':'BM_ReportingBUId','buname':'BranchName','UserID':'BM_UserID','UserName':'BM_UserName'}, inplace=True)
bro_df = df[(df['RoleId']==55)&(df['RoleName']=='BRO')][['U_BUID','UserID','UserName']]
bro_df.rename(columns={'U_BUID':'BRO_BUID','UserID':'BRO_UserID','UserName':'BRO_UserName'}, inplace=True)
M1 = pd.merge(bro_df,bm_df,left_on='BRO_BUID',right_on='BM_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M2 = pd.merge(M1,hub_df,left_on='BM_ReportingBUId',right_on='Hub_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M3 = pd.merge(M2,reg_df,left_on='Hub_ReportingBUId',right_on='Region_BUID',how='left').drop_duplicates(subset='BRO_UserID')
dF = pd.merge(M3,div_df,left_on='Region_ReportingBUId',right_on='Div_BUID',how='left').drop_duplicates(subset='BRO_UserID')

,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId


In [3]:
df[df['RoleName'] == 'BC Module']

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
44,2154,2,1,Varanasi,7555,Ankit Sharma,2154,43,BC Module
175,2201,3,2200,Hazaribagh RO,31085,VIGHNESWAR KUMAR YADAV,2201,43,BC Module


In [4]:
df[df['UserID'] == 19128].iloc[0]['buname']

'Varanasi'

In [5]:
reqdf = pd.merge(all_tasks_df, df, how='left', left_on='assigned_to', right_on='UserID').drop_duplicates(subset=['task_id'])
reqdf = pd.merge(reqdf, dF, how='left', left_on='UserID', right_on = 'BRO_UserID' )


In [94]:
reqdf[reqdf['Region_UserID'] == 7665]

KeyError: 'Region_UserID'

In [7]:
df.columns

Index(['BUId', 'BUType', 'ReportingBUId', 'buname', 'UserID', 'UserName',
       'U_BUID', 'RoleId', 'RoleName'],
      dtype='object')

In [8]:
type(reqdf['created_on'].iloc[0])

datetime.date

In [9]:
reqdf['priority'].value_counts()

1    48765
3       30
2       29
4       21
Name: priority, dtype: int64

In [10]:
reqdf[reqdf['priority'] == 2]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
20476,164374,test desc 2,Pending,2,31678,7535,None,2023-04-07,2023-04-09,1,None,None,6.0,5.0,553.0,Rudauli,31678.0,ABHISHEK DUBEY,6.0,55.0,BRO,6.0,31678.0,ABHISHEK DUBEY,6.0,Rudauli,1564.0,Virendra kumar Maurya,553.0,553.0,25094.0,ABHISHEK SINGH,FAIZABAD,1006.0,1006.0,18033.0,CHANDRA SHEKHAR,Faizabad,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow
20480,164378,test desc,Pending,2,31678,7535,None,2023-04-07,2023-04-09,1,None,None,6.0,5.0,553.0,Rudauli,31678.0,ABHISHEK DUBEY,6.0,55.0,BRO,6.0,31678.0,ABHISHEK DUBEY,6.0,Rudauli,1564.0,Virendra kumar Maurya,553.0,553.0,25094.0,ABHISHEK SINGH,FAIZABAD,1006.0,1006.0,18033.0,CHANDRA SHEKHAR,Faizabad,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow
20887,164789,test,Pending,2,30914,7535,None,2023-04-07,2023-04-20,14,None,None,17.0,5.0,586.0,Mishrikh,30914.0,AMRENDRA KUMAR GUPTA,17.0,55.0,BRO,17.0,30914.0,AMRENDRA KUMAR GUPTA,17.0,Mishrikh,15057.0,Surendra Kumar,586.0,586.0,7197.0,abhishek kumar singh,sitapur,2292.0,2292.0,0.0,REGION HEAD NOT POSTED,Sitapur,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow
22906,166812,test,Closed,2,30307,30307,2023-04-10,2023-04-10,2023-04-10,3,None,None,419.0,5.0,554.0,Dhaneypur,30307.0,Shiv Kumar,419.0,55.0,BRO,419.0,30307.0,Shiv Kumar,419.0,Dhaneypur,3988.0,Santosh Kumar,554.0,554.0,2399.0,Rakesh Kumar Pal,GONDA,1006.0,1006.0,18033.0,CHANDRA SHEKHAR,Faizabad,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow
23277,167183,test,Pending,2,7535,7535,None,2023-04-10,2023-04-10,3,None,None,651.0,2.0,1.0,Lucknow,7535.0,RAKESH KUMAR CHAUDHARY,651.0,34.0,DIVISION HEAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23332,167238,form,Pending,2,9739,9739,None,2023-04-10,2023-04-10,3,None,None,650.0,2.0,1.0,Prayagraj,9739.0,LAKSHMAN SINGH,650.0,34.0,DIVISION HEAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24690,168596,test,Closed,2,27784,27784,2023-04-11,2023-04-11,2023-04-12,15,2,1,419.0,5.0,554.0,Dhaneypur,27784.0,Suraj Kumar Yadav,419.0,55.0,BRO,419.0,27784.0,Suraj Kumar Yadav,419.0,Dhaneypur,3988.0,Santosh Kumar,554.0,554.0,2399.0,Rakesh Kumar Pal,GONDA,1006.0,1006.0,18033.0,CHANDRA SHEKHAR,Faizabad,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow
24789,168695,You have to describe task here whatever be the...,Re-Schedule,2,30823,30823,None,2023-04-11,2023-04-20,3,None,None,2244.0,5.0,2132.0,Aurai,30823.0,SAVITA,2244.0,55.0,BRO,2244.0,30823.0,SAVITA,2244.0,Aurai,1597.0,RAJU KUMAR MAURYA,2132.0,2132.0,3883.0,RAVINDRA KUMAR,HANDIA,1001.0,1001.0,7114.0,Akash Jyoti singh,Prayagraj,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
25013,168919,flejgejgsjg sgsgjgorjgorhun ahgiehghowihg,Pending,2,30823,30823,None,2023-04-11,2023-04-12,10,None,None,2244.0,5.0,2132.0,Aurai,30823.0,SAVITA,2244.0,55.0,BRO,2244.0,30823.0,SAVITA,2244.0,Aurai,1597.0,RAJU KUMAR MAURYA,2132.0,2132.0,3883.0,RAVINDRA KUMAR,HANDIA,1001.0,1001.0,7114.0,Akash Jyoti singh,Prayagraj,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
25199,169105,scs,Pending,2,30823,30823,None,2023-04-11,2023-04-26,13,None,None,2244.0,5.0,2132.0,Aurai,30823.0,SAVITA,2244.0,55.0,BRO,2244.0,30823.0,SAVITA,2244.0,Aurai,1597.0,RAJU KUMAR MAURYA,2132.0,2132.0,3883.0,RAVINDRA KUMAR,HANDIA,1001.0,1001.0,7114.0,Akash Jyoti singh,Prayagraj,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj


In [11]:
reqdf.columns

Index(['task_id', 'task_description', 'status', 'priority', 'assigned_to',
       'created_by', 'completion_date', 'created_on', 'deadline_date',
       'category', 'review', 'feedback', 'BUId', 'BUType', 'ReportingBUId',
       'buname', 'UserID', 'UserName', 'U_BUID', 'RoleId', 'RoleName',
       'BRO_BUID', 'BRO_UserID', 'BRO_UserName', 'BM_BUID', 'BranchName',
       'BM_UserID', 'BM_UserName', 'BM_ReportingBUId', 'Hub_BUID',
       'Hub_UserID', 'Hub_UserName', 'HubName', 'Hub_ReportingBUId',
       'Region_BUID', 'Region_UserID', 'Region_UserName', 'RegionName',
       'Region_ReportingBUId', 'Div_BUID', 'Div_UserID', 'Div_UserName',
       'DivName'],
      dtype='object')

In [95]:
dF

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,2325,27196,Mithilesh Kumar Paswan,2325,Nirmali,25630,Dharmveer Kumar,2104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
from datetime import datetime

In [14]:
type(datetime.now().date())

datetime.date

In [15]:
reqdf[reqdf['priority'] == int('1')]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Completed,1,29938,2271,2023-04-21,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO,459.0,29938.0,Virendra Singh gound,459.0,Burhar,4172.0,Akash Kumar Sen,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48840,192776,Customer (Customer id: 1356191) will pay an am...,Pending,1,31150,30855,None,2023-04-28,2023-05-01,3,None,None,492.0,5.0,609.0,Areraj,31150.0,KANHAIYA KUMAR,492.0,55.0,BRO,492.0,31150.0,KANHAIYA KUMAR,492.0,Areraj,21337.0,Ranjeet Bhagat,609.0,609.0,24774.0,MAHENDRA PRASAD MAURYA,Motihari,1020.0,1020.0,10850.0,Parasnath maorya,Motihari,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
48841,192777,Customer (Customer id: 3177937) will pay an am...,Pending,1,27611,3410,None,2023-04-28,2023-04-28,3,None,None,296.0,5.0,604.0,Nawada City,27611.0,Saurabh Kumar,296.0,55.0,BRO,296.0,27611.0,Saurabh Kumar,296.0,Nawada City,3410.0,Narendra Kumar Yadav,604.0,604.0,2554.0,Suresh Kumar,Nawada,1010.0,1010.0,4891.0,Praveen Singh,Gaya,2158.0,2158.0,15111.0,Ajay Kumar Singh,Patna
48842,192778,Customer (Customer id: Bha:Ala:348:223:015) ha...,Pending,1,31452,9731,None,2023-04-28,2023-04-28,10,None,None,348.0,5.0,573.0,Alwar City,31452.0,DILIP KUMAR YADAV,348.0,55.0,BRO,348.0,31452.0,DILIP KUMAR YADAV,348.0,Alwar City,23688.0,JALDEEP SINGH CHAUHAN,573.0,573.0,4505.0,Suraj Kumar,Alawar,1022.0,1022.0,7342.0,Neeraj maurya,Jaipur,2155.0,2155.0,7575.0,CHANDRA SHEKHAR,Jaipur
48843,192779,Customer (Customer id: Kan:Fat:40:123:015) has...,Pending,1,27790,27171,None,2023-04-28,2023-04-28,10,None,None,40.0,5.0,580.0,ghatampur,27790.0,Shivam Kumar,40.0,55.0,BRO,40.0,27790.0,Shivam Kumar,40.0

In [16]:
cols_req = [
        'created_on',
        'DivName',
        'RegionName',
        'HubName',
        'BranchName',
        'assigned_to',
        'created_by',
        'priority',
        'task_id',
        'category',
        'task_description',
        'deadline_date',
        #reschedule date
        #whatsapp conf
        'status',
        'completion_date',
        'review',
        'feedback'
    ]

In [17]:
reqdf = reqdf.sort_values(by=['created_on', 'DivName', 'RegionName', 'HubName', 'BranchName'], ascending=False)[cols_req]

In [18]:
reqdf

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
47742,2023-04-28,Varanasi,Varanasi,Varanasi,Varanasi city,16354,11905,1,191678,7,Customer (Customer id: Var:Var:2002:2614514) i...,2023-04-28,Pending,None,None,None
47701,2023-04-28,Varanasi,Varanasi,Varanasi,Pandeypur,31569,31569,1,191637,3,Customer (Customer id: 1224801) will pay an am...,2023-04-28,Pending,None,None,None
47702,2023-04-28,Varanasi,Varanasi,Varanasi,Pandeypur,31569,31569,1,191638,3,Customer (Customer id: 3060154) will pay an am...,2023-04-28,Pending,None,None,None
47703,2023-04-28,Varanasi,Varanasi,Varanasi,Pandeypur,31569,31569,1,191639,3,Customer (Customer id: 3060154) will pay an am...,2023-04-28,Pending,None,None,None
47704,2023-04-28,Varanasi,Varanasi,Varanasi,Pandeypur,31569,31569,1,191640,3,Customer (Customer id: 2176414) will pay an am...,2023-04-28,Pending,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,2023-04-01,NaN,NaN,NaN,NaN,25551,1473,1,129924,3,Customer (Customer id: Mot:Bet:2170:140:013) w...,2023-04-07,Pending,None,None,None
810,2023-04-01,NaN,NaN,NaN,NaN,27508,1473,1,129926,10,Customer (Customer id: Mot:Mot:477:035) has co...,2023-04-01,Pending,None,None,None
933,2023-04-01,NaN,NaN,NaN,NaN,31416,4785,1,130049,10,Customer (Customer id: Sag:DAM:262:2053211) ha...,2023-04-01,Pending,None,None,None
942,2023-04-01,NaN,NaN,NaN,NaN,24872,11903,1,130058,3,Customer (Customer id: Kau:KAR:297:159:008) wi...,2023-04-01,Pending,None,None,None


In [19]:
from_date = '2023-04-07'
to_date = '2023-04-13'

In [20]:
datetime.now().date()

datetime.date(2023, 4, 28)

In [21]:
datetime.strptime(from_date, "%Y-%M-%d").date() > datetime.now().date()

False

In [22]:
reqdf[(reqdf['created_on'] >= datetime.strptime(from_date, "%Y-%M-%d").date()) & (reqdf['created_on'] <= datetime.strptime(to_date, "%Y-%M-%d").date())]

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback


In [23]:
re1 = reqdf[(reqdf['created_on'] >= datetime.strptime(from_date, "%Y-%M-%d").date())]
re2 = re1[re1[(re1['created_on'] <= datetime.strptime(to_date, "%Y-%M-%d").date())]]

In [24]:
datetime.strptime(from_date, "%Y-%M-%d").date()

datetime.date(2023, 1, 7)

In [25]:
dF.columns

Index(['BRO_BUID', 'BRO_UserID', 'BRO_UserName', 'BM_BUID', 'BranchName',
       'BM_UserID', 'BM_UserName', 'BM_ReportingBUId', 'Hub_BUID',
       'Hub_UserID', 'Hub_UserName', 'HubName', 'Hub_ReportingBUId',
       'Region_BUID', 'Region_UserID', 'Region_UserName', 'RegionName',
       'Region_ReportingBUId', 'Div_BUID', 'Div_UserID', 'Div_UserName',
       'DivName'],
      dtype='object')

In [26]:
reqdf[(reqdf['deadline_date'] >= datetime.strptime(from_date, "%Y-%m-%d").date()) & (reqdf['deadline_date'] <= datetime.strptime(to_date, "%Y-%m-%d").date())]


,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
46464,2023-04-27,Muzaffarpur,Samastipur,Begusarai,Manjahaul,30435,13048,1,190400,3,Customer (Customer id: 1155279) will pay an am...,2023-04-07,Pending,None,None,None
43591,2023-04-25,Saharanpur,Karnal,Ladwa,Vilashpur,21371,21371,1,187527,3,Customer (Customer id: 2772067) will pay an am...,2023-04-13,Completed,2023-04-25,None,None
41701,2023-04-24,Prayagraj,Sultanpur,AKBARPUR,Bhiti,32021,3512,1,185637,7,"Mr/Ms Sheela ,Abd:Bti:01:090:009 is a good cu...",2023-04-07,Pending,None,None,None
38309,2023-04-20,Prayagraj,Kaushambi,BHARWARI,Bharawari,28207,28207,1,182245,3,Customer (Customer id: 3228278) will pay an am...,2023-04-12,Pending,None,None,None
33069,2023-04-17,Saharanpur,Karnal,Ladwa,Yamuna Nagar,31908,31908,1,177005,3,Customer (Customer id: 3430029) will pay an am...,2023-04-13,Completed,2023-04-18,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,2023-04-01,NaN,NaN,NaN,NaN,30788,1473,1,129775,3,Customer (Customer id: Mot:GOP:2057:146:010) w...,2023-04-07,Pending,None,None,None
682,2023-04-01,NaN,NaN,NaN,NaN,26917,1473,1,129798,3,Customer (Customer id: Mot:Mot:477:185:010) wi...,2023-04-07,Pending,None,None,None
738,2023-04-01,NaN,NaN,NaN,NaN,25551,1473,1,129854,3,Customer (Customer id: Mot:Bet:2170:137:015) w...,2023-04-07,Pending,None,None,None
799,2023-04-01,NaN,NaN,NaN,NaN,26917,1473,1,129915,3,Customer (Customer id: Ech:Cka:01:039:028) wil...,2023-04-07,Pending,None,None,None


In [27]:
reqdf[reqdf['priority'] == 3]

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
48028,2023-04-28,NaN,NaN,NaN,NaN,7082,7082,3,191964,Choose the Category,testing,2023-04-30,Pending,None,None,None
44501,2023-04-26,Muzaffarpur,Motihari,Motihari,Barauli,28281,17830,3,188437,12,Customer Meera Devi Want a New Loan Of Amount...,2023-05-17,Pending,None,None,None
42872,2023-04-25,Jabalpur,Katni,Burhar,Kotma,30774,30774,3,186808,12,Customer Gulabiya Bai Want a New Loan Of Amoun...,2023-05-10,Pending,None,None,None
38411,2023-04-20,Varanasi,Varanasi,Varanasi,Varanasi city,7173,7463,3,182347,7,20,2023-04-20,Completed,2023-04-20,None,None
37435,2023-04-20,Jabalpur,Nagpur,Nagpur,Pulgaon,30731,4749,3,181371,3,Aaj kist dete hai,2023-04-20,Pending,None,None,None
33943,2023-04-18,Prayagraj,Sultanpur,AKBARPUR,Alapur,3401,2163,3,177879,Choose the Category,test,2023-04-29,Completed,2023-04-18,None,None
32465,2023-04-17,Jabalpur,Jabalpur,NARSINGHPUR,Tendukheda,30808,30808,3,176401,12,Customer mayabaee Want a New Loan Of Amount . ...,2023-06-08,Completed,2023-04-25,None,None
31582,2023-04-15,Varanasi,Varanasi,Varanasi,Pandeypur,30255,4017,3,175518,2,10 client disbursement Ese hr hal me pura krna...,2023-04-15,Completed,2023-04-18,None,None
31575,2023-04-14,Jabalpur,Jabalpur,GADARWARA,Bareli MP,30161,11958,3,175511,3,\r\n\r\n,2023-04-14,Completed,2023-04-24,None,None
30973,2023-04-14,NaN,NaN,NaN,NaN,7082,7082,3,174909,5,dkhfhfoewf jbfhfihfuh khfhefhe fkehfuewhO BEWV...,2023-04-27,Pending,None,None,None


In [28]:
d = dict(zip(df['UserID'], df['UserName']))
reqdf['assigned_to'] = reqdf['assigned_to'].apply(lambda x : str(x) + '-' + d[x] if x in d.keys() else str(x))

In [29]:
div_df[div_df['Div_UserID'] == 9739]['DivName'].iloc[0]

'Prayagraj'

In [30]:
df[df['UserID'] == 9739]['buname'].iloc[0]

'Prayagraj'

In [31]:
df[df['UserID'] == 1074].iloc[0]['buname']

'Bharatpur'

In [32]:
reqdf[(reqdf['BranchName'] == 'Katangi')]

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
45766,2023-04-27,Jabalpur,CHINDWADA,SEONI,Katangi,31545-VISHAL BISEN,29968,1,189702,7,Customer (Customer id: CHI:SEO:2127:064:012) i...,2023-04-27,Pending,None,None,None
46423,2023-04-27,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,1858,1,190359,3,Customer (Customer id: 2210573) will pay an am...,2023-04-29,Pending,None,None,None
46432,2023-04-27,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,1858,1,190368,3,Customer (Customer id: 2225744) will pay an am...,2023-04-29,Pending,None,None,None
46462,2023-04-27,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,1858,1,190398,3,Customer (Customer id: 3210104) will pay an am...,2023-05-03,Pending,None,None,None
46472,2023-04-27,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,29968,1,190408,10,Customer (Customer id: CHI:SEO:2127:008:014) h...,2023-04-27,Pending,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2023-04-01,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,29968,1,129200,3,Customer (Customer id: CHI:SEO:2127:051:006) w...,2023-04-04,Pending,None,None,None
120,2023-04-01,Jabalpur,CHINDWADA,SEONI,Katangi,31545-VISHAL BISEN,29968,1,129236,3,Customer (Customer id: CHI:SEO:2127:070:011) w...,2023-04-05,Pending,None,None,None
129,2023-04-01,Jabalpur,CHINDWADA,SEONI,Katangi,31545-VISHAL BISEN,29968,1,129245,3,Customer (Customer id: CHI:SEO:2127:011:010) w...,2023-04-04,Pending,None,None,None
323,2023-04-01,Jabalpur,CHINDWADA,SEONI,Katangi,31545-VISHAL BISEN,29968,1,129439,10,Customer (Customer id: CHI:SEO:2127:013) has c...,2023-04-01,Pending,None,None,Wrong number Sahi number update Karwaiye


In [33]:
new_columns = ['Assigned_Date','Division','Region','Hub', 'Branch', 'Assigned_To_User', 'Assigned_By', 'Priority', 'TaskID', 'Category', 'Task_Description', 'Target_Date', 'Status', 'Close_Date', 'Review', 'Feedback']
len(new_columns)

16

In [34]:
len(reqdf.columns)

16

In [35]:
dF.columns

Index(['BRO_BUID', 'BRO_UserID', 'BRO_UserName', 'BM_BUID', 'BranchName',
       'BM_UserID', 'BM_UserName', 'BM_ReportingBUId', 'Hub_BUID',
       'Hub_UserID', 'Hub_UserName', 'HubName', 'Hub_ReportingBUId',
       'Region_BUID', 'Region_UserID', 'Region_UserName', 'RegionName',
       'Region_ReportingBUId', 'Div_BUID', 'Div_UserID', 'Div_UserName',
       'DivName'],
      dtype='object')

In [36]:
dF['RegionName'].unique()

array([nan, 'Kaushambi', 'Sultanpur', 'Faizabad', 'Lucknow', 'Sitapur',
       'Prayagraj', 'Sagar', 'Kanpur', 'Bharatpur', 'Rewa', 'Bareilly',
       'Moradabad', 'Katni', 'Jabalpur', 'Saharanpur', 'Karnal',
       'CHINDWADA', 'Ghaziabad', 'Gaya', 'Muzaffarpur', 'Gwalior',
       'Samastipur', 'Jaipur', 'Jaunpur', 'Varanasi', 'Chandigarh',
       'Gorakhpur', 'Deoria', 'Motihari', 'Nagpur', 'Ghazipur RO',
       'Darbhanga', 'BUXAR', 'PATNA', 'Hazaribagh RO', 'Giridih',
       'Ranchi', 'Rajnandgaon', 'Raipur'], dtype=object)

In [37]:
dF['Region_UserID'].unique()

array([   nan, 18044.,  4347., 18033.,     0.,  7114., 18024., 18043.,
       15054., 18023.,  7664.,  1968.,  1622.,  1174., 18034.,  4708.,
       17692., 14050.,  4891., 24776., 16347., 18035.,  7342.,  7074.,
       10850., 24808., 21479., 10848., 23657., 18036., 17841., 21376.])

In [38]:
dF[dF['BM_UserID'] == 7082]

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
958,307,3401,Nagendra Pratap Singh,307.0,Alapur,7082.0,Surendra yadav,551.0,551.0,7660.0,Dharmendra Pratap Singh,AKBARPUR,1024.0,1024.0,4347.0,SHIV BAHADUR,Sultanpur,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
959,307,4874,Sanjay Kumar Bind,307.0,Alapur,7082.0,Surendra yadav,551.0,551.0,7660.0,Dharmendra Pratap Singh,AKBARPUR,1024.0,1024.0,4347.0,SHIV BAHADUR,Sultanpur,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
960,307,5079,Sudhir Singh,307.0,Alapur,7082.0,Surendra yadav,551.0,551.0,7660.0,Dharmendra Pratap Singh,AKBARPUR,1024.0,1024.0,4347.0,SHIV BAHADUR,Sultanpur,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
961,307,17789,VIKAS KUMAR,307.0,Alapur,7082.0,Surendra yadav,551.0,551.0,7660.0,Dharmendra Pratap Singh,AKBARPUR,1024.0,1024.0,4347.0,SHIV BAHADUR,Sultanpur,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
962,307,17857,SANJAY KUMAR VISHWAKARMA,307.0,Alapur,7082.0,Surendra yadav,551.0,551.0,7660.0,Dharmendra Pratap Singh,AKBARPUR,1024.0,1024.0,4347.0,SHIV BAHADUR,Sultanpur,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
963,307,29606,RANJEET KUMAR,307.0,Alapur,7082.0,Surendra yadav,551.0,551.0,7660.0,Dharmendra Pratap Singh,AKBARPUR,1024.0,1024.0,4347.0,SHIV BAHADUR,Sultanpur,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj


In [39]:
hub_df[hub_df['Hub_UserID'] ==7082]

,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId


In [40]:
bm_df[bm_df['BM_UserID'] ==7082]

,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId
1531,307,Alapur,7082,Surendra yadav,551


In [41]:
all_tasks_df[all_tasks_df['status'] == 'Completed']

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Completed,1,29938,2271,2023-04-21,2023-04-01,2023-04-01,7,None,None
17,129133,Customer (Customer id: Mbd:Chd:03:316) has com...,Completed,1,31693,30892,2023-04-11,2023-04-01,2023-04-01,10,None,Wrong number
26,129142,Customer (Customer id: Mbd:Chd:03:280) is inte...,Completed,1,31693,30892,2023-04-11,2023-04-01,2023-04-01,7,None,None
33,129149,Customer (Customer id: Swn:Swn:01:039:035) wil...,Completed,1,31187,29968,2023-04-15,2023-04-01,2023-04-12,3,None,None
37,129153,Customer (Customer id: Swn:Swn:01:010:017) wil...,Completed,1,31184,29968,2023-04-13,2023-04-01,2023-04-07,3,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
47993,191929,Customer (Customer id: Har:Ror:01:016:027) is ...,Completed,1,28018,27168,2023-04-28,2023-04-28,2023-04-28,7,None,None
48269,192205,Subeena Bano (Customer id: 3126736) is intere...,Completed,1,16469,27199,2023-04-28,2023-04-28,2023-06-19,7,None,None
48310,192246,Customer (Customer id: 2540146) has paid the l...,Completed,1,31063,27199,2023-04-28,2023-04-28,2023-04-30,3,None,None
48378,192314,Customer (Customer id: 2540146) has paid the l...,Completed,1,31063,27199,2023-04-28,2023-04-28,2023-05-30,3,None,None


In [42]:
reg_df['RegionName'].unique()

array(['Lucknow', 'Jabalpur', 'Varanasi', 'Prayagraj', 'Kanpur',
       'Saharanpur', 'Moradabad', 'Faizabad', 'Bharatpur', 'Gaya',
       'Ghaziabad', 'Muzaffarpur', 'Gwalior', 'Samastipur', 'Sagar',
       'Satna', 'Chandigarh', 'Gorakhpur', 'Motihari', 'Nagpur', 'Jaipur',
       'CHINDWADA', 'Sultanpur', 'Ghazipur RO', 'BUXAR', 'PATNA',
       'Karnal', 'Bareilly', 'Kaushambi', 'Rewa', 'Katni', 'Darbhanga',
       'Hazaribagh RO', 'Giridih', 'Ranchi', 'Raipur', 'Rajnandgaon',
       'Jaunpur', 'Deoria', 'Sitapur'], dtype=object)

In [43]:
df.tail(-1)

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
2,650,2,1,Prayagraj,9739,LAKSHMAN SINGH,650,34,DIVISION HEAD
3,650,2,1,Prayagraj,9739,LAKSHMAN SINGH,650,44,Credit Committee
4,650,2,1,Prayagraj,15183,ABDUL KADIR ANSARI,650,58,Audit
5,650,2,1,Prayagraj,16278,Dheeraj Shukla,650,39,Disbursement Confirmer
6,650,2,1,Prayagraj,19113,MUKESH YADAV,650,58,Audit
...,...,...,...,...,...,...,...,...,...
3006,2326,5,2206,Chandwa,27375,Rahul Raj,2326,13,Branch Manager
3007,2326,5,2206,Chandwa,31817,MODASSHIR RAZA,2326,55,BRO
3008,2327,5,607,Kurha,8699,SHASHIDHAR PASWAN,2327,13,Branch Manager
3009,2327,5,607,Kurha,31763,SAURABH KUMAR,2327,55,BRO


In [44]:
dF.head()

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,591,4829,ANKIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3353,RAHUL KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
2,2,25715,DHEERAJ KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
3,2,27274,Saurabh Kushwaha,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj
4,2,31073,NEERAJ KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj


In [45]:
def get_region(userid):
    return dF[dF['BRO_UserID'] == userid][['Region_UserID', 'RegionName']]

In [46]:
get_region(25715)

,Region_UserID,RegionName
2,18044.0,Kaushambi


In [47]:
from datetime import date, datetime, timedelta

In [48]:
str(date.today() - timedelta(days=1))

'2023-04-27'

In [49]:
list(dF['BRO_UserID'])

[4829,
 3353,
 25715,
 27274,
 31073,
 4047,
 7219,
 31572,
 31574,
 7408,
 27892,
 30533,
 31935,
 32020,
 1478,
 3120,
 3529,
 28408,
 29881,
 29903,
 30616,
 31678,
 31787,
 4741,
 16263,
 30820,
 31568,
 31655,
 27179,
 27245,
 29995,
 22484,
 25555,
 30160,
 30308,
 30399,
 5219,
 16239,
 25549,
 30552,
 4183,
 9760,
 21310,
 27973,
 30890,
 31224,
 31849,
 4726,
 22615,
 30218,
 31575,
 28446,
 30914,
 31001,
 3661,
 29527,
 30754,
 17728,
 23722,
 30168,
 30302,
 2916,
 7138,
 30185,
 31109,
 4913,
 30530,
 31593,
 435,
 25428,
 30901,
 27674,
 29702,
 30515,
 31337,
 31766,
 32102,
 32103,
 25137,
 29528,
 29834,
 3811,
 17585,
 26818,
 32097,
 25292,
 29654,
 30323,
 30778,
 31120,
 32093,
 25400,
 29830,
 31064,
 31489,
 31687,
 27460,
 28337,
 31488,
 31717,
 32009,
 5241,
 21287,
 30362,
 31382,
 22567,
 27786,
 30306,
 31547,
 31865,
 31976,
 32084,
 27083,
 27108,
 30143,
 31406,
 31554,
 31647,
 32046,
 3572,
 21224,
 26871,
 28008,
 32089,
 15077,
 26979,
 28001,
 29481

In [50]:
bro_df

,BRO_BUID,BRO_UserID,BRO_UserName
244,591,4829,ANKIT
361,2,3353,RAHUL KUMAR
363,2,25715,DHEERAJ KUMAR
364,2,27274,Saurabh Kushwaha
365,2,31073,NEERAJ KUMAR
...,...,...,...
3002,2324,31527,ANNU KUSHWAHA
3003,2324,31751,SAKSHI CHOUDHARY
3005,2325,27196,Mithilesh Kumar Paswan
3007,2326,31817,MODASSHIR RAZA


In [51]:
df[df['UserID'] == 25630]

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
3004,2325,5,2104,Nirmali,25630,Dharmveer Kumar,2325,13,Branch Manager


In [52]:
dF[dF['BranchName'] == 'Nirmali']

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
2096,2325,27196,Mithilesh Kumar Paswan,2325.0,Nirmali,25630.0,Dharmveer Kumar,2104.0,2104.0,3904.0,RANDHEER KUMAR PASWAN,Madhubani,2198.0,2198.0,10848.0,SHYAM KISHOR MISHRA,Darbhanga,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur


In [53]:
dF[dF['BM_UserID'] == 25630]['BranchName'].iloc[0]

'Nirmali'

In [54]:
branch_names = bm_df['BranchName'].unique()
branch_names

array(['Giridih', 'Sirathu', 'Nari Bari', 'Akbarpur', 'Tanda', 'Rudauli',
       'Sohawal', 'Barabanki city', 'Sidhauli', 'Malihabad', 'Mahmudabad',
       'Sitapur city', 'Itaunja', 'Mishrikh', 'Maharaj ganj',
       'Pratap Garh', 'Derwa', 'Gosaiganj', 'patti', 'Garhakota',
       'Akberpur', 'chaubey pur', 'Hathrash', 'Jalesar', 'Sikohabad',
       'Kumar ganj', 'Rewa City', 'Bareilly', 'Dostpur', 'Puramufti',
       'Lambhua', 'Maihar', 'ghatampur', 'Gohaniya', 'Soraw',
       'Moradabad City', 'Bilari', 'Chandpur', 'Joya', 'Mangawan',
       'Basti', 'Rani Ganj', 'Manikpur', 'Mahuli', 'Hariya', 'Rudhauli',
       'Satna City', 'Saray akill', 'Hanumanganj', 'Kadipur',
       'Mosafir khana', 'KATNI', 'Bhadevara', 'Sagar', 'kudhebhaar',
       'Gadarwara', 'Lalganj', 'DurgaGanj', 'GOTEGAON', 'Garha',
       'Karchhana', 'Badaun', 'Billhor', 'Bilsi', 'Faizabad', 'Raibareli',
       'Mauaima', 'PANAGAR', 'Faridpur', 'Majholi', 'Gauriganj', 'RANJHI',
       'Gwari Ghat', 'Salon', 'Fate

In [55]:
bro_df

,BRO_BUID,BRO_UserID,BRO_UserName
244,591,4829,ANKIT
361,2,3353,RAHUL KUMAR
363,2,25715,DHEERAJ KUMAR
364,2,27274,Saurabh Kushwaha
365,2,31073,NEERAJ KUMAR
...,...,...,...
3002,2324,31527,ANNU KUSHWAHA
3003,2324,31751,SAKSHI CHOUDHARY
3005,2325,27196,Mithilesh Kumar Paswan
3007,2326,31817,MODASSHIR RAZA


In [56]:
df[(df['buname'] == 'Roorkee') & (df['RoleName'] == 'BRO')]

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
1122,176,5,600,Roorkee,28018,RAJNEESH KUMAR,176,55,BRO
1123,176,5,600,Roorkee,30928,TEJPAL,176,55,BRO
1124,176,5,600,Roorkee,31278,ANKUR KUMAR,176,55,BRO
1125,176,5,600,Roorkee,31470,AAKASH KUMAR,176,55,BRO
1126,176,5,600,Roorkee,31928,ANUJ KUMAR,176,55,BRO
1127,176,5,600,Roorkee,32012,AAKSAH KUMAR,176,55,BRO


In [57]:
df['buname'].unique()

array(['Prayagraj', 'Lucknow', 'Jabalpur', 'Muzaffarpur', 'Moradabad',
       'Varanasi', 'Jaipur', 'Gwalior', 'Saharanpur', 'Patna',
       'Ranchi DO', 'Raipur', 'Kanpur', 'Faizabad', 'Bharatpur', 'Gaya',
       'Ghaziabad', 'Samastipur', 'Sagar', 'Satna', 'Chandigarh',
       'Gorakhpur', 'Motihari', 'Nagpur', 'CHINDWADA', 'Sultanpur',
       'Ghazipur RO', 'BUXAR', 'PATNA', 'Karnal', 'Bareilly', 'Kaushambi',
       'Rewa', 'Katni', 'Darbhanga', 'Hazaribagh RO', 'Giridih', 'Ranchi',
       'Rajnandgaon', 'Jaunpur', 'Deoria', 'Sitapur', 'BHARWARI',
       'City Branch', 'KARCHANA', 'KARVI', 'PHOOLPUR', 'PRATAPGARH',
       'AKBARPUR', 'BASTI', 'FAIZABAD', 'GONDA', 'KADIPUR', 'SULTANPUR',
       'Ghazipur Hub', 'GWALIOR', 'GADARWARA', 'JABALPUR URBAN', 'MANDLA',
       'SIHORA', 'DAMOH', 'PANNA', 'SAGAR', 'KATNI', 'REWA', 'Alawar',
       'Hathrash', 'Mathura', 'Akberpur', 'Billaur', 'Fatehpur',
       'Barabanki', 'Raibarely', 'Sidhauli', 'sitapur', 'Patiaya City',
       'Hapur', 'B

In [58]:
sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 25630;SET NOCOUNT OFF"""
nbdf=pd.read_sql_query(sql_query,engine)


In [59]:
nbdf

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
0,2325,5,2104,Nirmali,25630,Dharmveer Kumar,2325,13,Branch Manager
1,2325,5,2104,Nirmali,27196,Mithilesh Kumar Paswan,2325,55,BRO


In [60]:
div_df = nbdf[(nbdf['RoleId']==34)&(nbdf['RoleName']=='DIVISION HEAD')][['U_BUID','UserID','UserName','buname']]
div_df.rename(columns={'U_BUID':'Div_BUID','buname':'DivName','UserID':'Div_UserID','UserName':'Div_UserName'}, inplace=True)
reg_df = nbdf[(nbdf['RoleId']==35)&(nbdf['RoleName']=='REGION HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
reg_df.rename(columns={'U_BUID':'Region_BUID','ReportingBUId':'Region_ReportingBUId','buname':'RegionName','UserID':'Region_UserID','UserName':'Region_UserName'}, inplace=True)
hub_df = nbdf[(nbdf['RoleId']==36)&(nbdf['RoleName']=='HUB HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
hub_df.rename(columns={'U_BUID':'Hub_BUID','ReportingBUId':'Hub_ReportingBUId','buname':'HubName','UserID':'Hub_UserID','UserName':'Hub_UserName'}, inplace=True)
bm_df = nbdf[(nbdf['RoleId']==13)&(nbdf['RoleName']=='Branch Manager')][['U_BUID','buname','UserID','UserName','ReportingBUId']]
bm_df.rename(columns={'U_BUID':'BM_BUID','ReportingBUId':'BM_ReportingBUId','buname':'BranchName','UserID':'BM_UserID','UserName':'BM_UserName'}, inplace=True)
bro_df = nbdf[(nbdf['RoleId']==55)&(nbdf['RoleName']=='BRO')][['U_BUID','UserID','UserName']]
bro_df.rename(columns={'U_BUID':'BRO_BUID','UserID':'BRO_UserID','UserName':'BRO_UserName'}, inplace=True)
M1 = pd.merge(bro_df,bm_df,left_on='BRO_BUID',right_on='BM_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M2 = pd.merge(M1,hub_df,left_on='BM_ReportingBUId',right_on='Hub_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M3 = pd.merge(M2,reg_df,left_on='Hub_ReportingBUId',right_on='Region_BUID',how='left').drop_duplicates(subset='BRO_UserID')
dF = pd.merge(M3,div_df,left_on='Region_ReportingBUId',right_on='Div_BUID',how='left').drop_duplicates(subset='BRO_UserID')

In [61]:
dF

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,2325,27196,Mithilesh Kumar Paswan,2325,Nirmali,25630,Dharmveer Kumar,2104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
reqdf = pd.merge(all_tasks_df, nbdf, how='left', left_on='assigned_to', right_on='UserID').drop_duplicates(subset=['task_id'])
reqdf = pd.merge(reqdf, dF, how='left', left_on='UserID', right_on = 'BRO_UserID' )

In [63]:
reqdf[reqdf['assigned_to'] == 25630]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName


In [64]:
sql_query = f"""SET NOCOUNT ON;select * from dbo.TaskCategoryMasterTable;SET NOCOUNT OFF"""
category_df = pd.read_sql_query(sql_query,engine)
category_df

,CategoryID,CategoryName
0,3,Collection Followup
1,4,Data Correction
2,5,Proposal Followup
3,6,Promise to Pay
4,7,Retention Followup
5,8,Post Disbursement
6,9,Pre Disbursement
7,10,Wrong Number
8,11,Center Visit Compliance
9,12,Branch Visit Compliance


In [65]:
category_df.dtypes

CategoryID       int64
CategoryName    object
dtype: object

In [66]:
category_dict = dict(zip(category_df['CategoryID'], category_df['CategoryName']))
category_dict

{3: 'Collection Followup',
 4: 'Data Correction',
 5: 'Proposal Followup',
 6: 'Promise to Pay',
 7: 'Retention Followup',
 8: 'Post Disbursement',
 9: 'Pre Disbursement',
 10: 'Wrong Number',
 11: 'Center Visit Compliance',
 12: 'Branch Visit Compliance',
 13: 'BC Application Correction',
 14: 'General',
 15: 'NACH Bounce',
 2: 'DisbursementID',
 1: 'CenterTask',
 16: 'Customer Followup'}

In [67]:
cols_req = [
        'created_on',
        'DivName',
        'RegionName',
        'HubName',
        'BranchName',
        'assigned_to',
        'created_by',
        'priority',
        'task_id',
        'category',
        'task_description',
        'deadline_date',
        #reschedule date
        #whatsapp conf
        'status',
        'completion_date',
        'review',
        'feedback'
    ]

In [68]:
reqdf = reqdf[cols_req]
reqdf

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
0,2023-04-01,NaN,NaN,NaN,NaN,29938,2271,1,129116,7,Customer (Customer id: Sdl:Bhr:01:006) is inte...,2023-04-01,Completed,2023-04-21,None,None
1,2023-04-01,NaN,NaN,NaN,NaN,30482,27199,1,129117,10,Customer (Customer id: Rab:cit:01:236:037) has...,2023-04-01,Pending,None,None,None
2,2023-04-01,NaN,NaN,NaN,NaN,27245,27199,1,129118,10,Customer (Customer id: Bab:cit:01:189:043) has...,2023-04-01,Pending,None,None,None
3,2023-04-01,NaN,NaN,NaN,NaN,27815,27815,1,129119,3,Customer (Customer id: 1397268) will pay an am...,2023-04-01,Pending,None,None,None
4,2023-04-01,NaN,NaN,NaN,NaN,31063,27199,1,129120,10,Customer (Customer id: Lko:mhn:01:116:004) has...,2023-04-01,Pending,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48840,2023-04-28,NaN,NaN,NaN,NaN,31150,30855,1,192776,3,Customer (Customer id: 1356191) will pay an am...,2023-05-01,Pending,None,None,None
48841,2023-04-28,NaN,NaN,NaN,NaN,27611,3410,1,192777,3,Customer (Customer id: 3177937) will pay an am...,2023-04-28,Pending,None,None,None
48842,2023-04-28,NaN,NaN,NaN,NaN,31452,9731,1,192778,10,Customer (Customer id: Bha:Ala:348:223:015) ha...,2023-04-28,Pending,None,None,None
48843,2023-04-28,NaN,NaN,NaN,NaN,27790,27171,1,192779,10,Customer (Customer id: Kan:Fat:40:123:015) has...,2023-04-28,Pending,None,None,None


In [69]:
category_dict[8]
type(list(category_dict.keys())[0])

int

In [70]:
reqdf['category'].apply(lambda x : str(x) + '-' + category_dict[int(x)] if (x.isdigit() and (int(x) in category_dict.keys())) else str(x))


0         7-Retention Followup
1              10-Wrong Number
2              10-Wrong Number
3        3-Collection Followup
4              10-Wrong Number
                 ...          
48840    3-Collection Followup
48841    3-Collection Followup
48842          10-Wrong Number
48843          10-Wrong Number
48844    3-Collection Followup
Name: category, Length: 48845, dtype: object

In [71]:
reqdf['category'].apply(lambda x : str(x) if x == 'Choose the Category' else str(x) + '-' + category_dict[int(x)])

0         7-Retention Followup
1              10-Wrong Number
2              10-Wrong Number
3        3-Collection Followup
4              10-Wrong Number
                 ...          
48840    3-Collection Followup
48841    3-Collection Followup
48842          10-Wrong Number
48843          10-Wrong Number
48844    3-Collection Followup
Name: category, Length: 48845, dtype: object

In [72]:
type(reqdf['category'])

pandas.core.series.Series

In [73]:
all_tasks_df['category'].apply(lambda x : (x.isdigit() == False))

0        False
1        False
2        False
3        False
4        False
         ...  
48840    False
48841    False
48842    False
48843    False
48844    False
Name: category, Length: 48845, dtype: bool

In [74]:
all_tasks_df[all_tasks_df['category'].apply(lambda x : (x.isdigit() == False)) ]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback
20630,164529,test dssxc,Pending,3,18091,7535,None,2023-04-07,2023-04-21,Choose the Category,None,None
23247,167153,khascvksehbvksv,Pending,4,27784,27784,None,2023-04-10,2023-04-20,Choose the Category,None,None
33803,177739,test,Pending,1,17789,2163,None,2023-04-18,2023-04-20,Choose the Category,None,None
33943,177879,test,Completed,3,3401,2163,2023-04-18,2023-04-18,2023-04-29,Choose the Category,None,None
34138,178074,Testing,Pending,2,3401,2163,None,2023-04-18,2023-05-05,Choose the Category,None,None
34155,178091,testing Description,Pending,2,17789,2163,None,2023-04-18,2023-04-27,Choose the Category,None,None
35801,179737,testing assign a new task to BM,Pending,2,2164,7082,None,2023-04-19,2023-04-28,Choose the Category,None,None
48028,191964,testing,Pending,3,7082,7082,None,2023-04-28,2023-04-30,Choose the Category,None,None


In [75]:
reqdf[reqdf['category'] == 'Choose the Category']

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
20630,2023-04-07,NaN,NaN,NaN,NaN,18091,7535,3,164529,Choose the Category,test dssxc,2023-04-21,Pending,None,None,None
23247,2023-04-10,NaN,NaN,NaN,NaN,27784,27784,4,167153,Choose the Category,khascvksehbvksv,2023-04-20,Pending,None,None,None
33803,2023-04-18,NaN,NaN,NaN,NaN,17789,2163,1,177739,Choose the Category,test,2023-04-20,Pending,None,None,None
33943,2023-04-18,NaN,NaN,NaN,NaN,3401,2163,3,177879,Choose the Category,test,2023-04-29,Completed,2023-04-18,None,None
34138,2023-04-18,NaN,NaN,NaN,NaN,3401,2163,2,178074,Choose the Category,Testing,2023-05-05,Pending,None,None,None
34155,2023-04-18,NaN,NaN,NaN,NaN,17789,2163,2,178091,Choose the Category,testing Description,2023-04-27,Pending,None,None,None
35801,2023-04-19,NaN,NaN,NaN,NaN,2164,7082,2,179737,Choose the Category,testing assign a new task to BM,2023-04-28,Pending,None,None,None
48028,2023-04-28,NaN,NaN,NaN,NaN,7082,7082,3,191964,Choose the Category,testing,2023-04-30,Pending,None,None,None


In [76]:
reqdf['assigned_to']

0        29938
1        30482
2        27245
3        27815
4        31063
         ...  
48840    31150
48841    27611
48842    31452
48843    27790
48844    28450
Name: assigned_to, Length: 48845, dtype: int64

In [77]:
df[df['UserID'] == 7082]

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
1531,307,5,551,Alapur,7082,Surendra yadav,307,13,Branch Manager


In [78]:
sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 6892;SET NOCOUNT OFF"""
df=pd.read_sql_query(sql_query,engine)

In [79]:
reqdf['priority'].replace(1, 'Immediate')

0        Immediate
1        Immediate
2        Immediate
3        Immediate
4        Immediate
           ...    
48840    Immediate
48841    Immediate
48842    Immediate
48843    Immediate
48844    Immediate
Name: priority, Length: 48845, dtype: object

In [80]:
len(reqdf)

48845

In [81]:
reqdf.to_csv('reqdfcsv.csv')

In [82]:
dF.to_csv('full_UHD.csv')

In [83]:
df.to_csv('UHD.csv')

In [84]:
df[df['UserID'] == 7665]

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
61,2155,2,1,Jaipur,7665,Krishna Vijay Singh,2155,58,Audit


In [85]:
df[df['buname'] == 'Moradabad']

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
35,654,2,1,Moradabad,7662,Sandeep Singh,654,34,DIVISION HEAD
36,654,2,1,Moradabad,7662,Sandeep Singh,654,44,Credit Committee
37,654,2,1,Moradabad,17560,Rashmi kumari,654,39,Disbursement Confirmer
38,654,2,1,Moradabad,19109,MAYANK PANDEY,654,58,Audit
39,654,2,1,Moradabad,23725,Rajnath Yadav,654,59,Risk manager
40,654,2,1,Moradabad,28117,SUNIL KUMAR,654,64,Verification Head
41,654,2,1,Moradabad,31995,PRINCE GAUTAM,654,58,Audit
101,1005,3,654,Moradabad,1968,sompal,1005,35,REGION HEAD
102,1005,3,654,Moradabad,31303,PRADEEP KUMAR,1005,58,Audit
246,592,4,1005,Moradabad,1965,sanjeew,592,36,HUB HEAD


In [86]:
user1 = os.environ['PAYMEE_SONTATA_DB_USER']
password = os.environ['PAYMEE_SONTATA_DB_PASSWORD']
db = "Sonata_Connect"
server = "172.17.130.216"
engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")
sql_query = f"""SET NOCOUNT ON;EXEC [Sonata_Connect].[dbo].[CSP_STAGE_RESOLUTION] @userid = {7665};SET NOCOUNT OFF"""
sp_df=pd.read_sql_query(sql_query,engine)
userid_list = sp_df['USERID'].unique().tolist()
userid_list

[10803,
 10804,
 25320,
 29571,
 29609,
 29635,
 29654,
 29706,
 29830,
 31006,
 31044,
 31064,
 31452,
 31488,
 31489,
 31495,
 17658,
 17808,
 26882,
 27024,
 29983,
 30006,
 30022,
 31120,
 31122,
 31830,
 31875,
 31924,
 31927,
 4915,
 23734,
 28041,
 30663,
 30716,
 30726,
 30751,
 30778,
 30779,
 30832,
 30863,
 30865,
 31591,
 31594,
 31595,
 31596,
 31618,
 31684,
 31685,
 31687,
 31688,
 31717,
 24790,
 24947,
 25292,
 28185,
 28310,
 28337,
 28396,
 28416,
 28455,
 30450,
 30514,
 17967,
 21155,
 27460,
 27504,
 27534,
 27758,
 30155,
 30323,
 30331,
 31264,
 31946,
 32008,
 32013,
 32033]

In [87]:
def region_checker(rgn_id,user_id):
    """This function checks if a particular user belongs to the TEST region. To be used for WhatsAppBot Beta Testing"""
   
    user1 = os.environ['PAYMEE_SONTATA_DB_USER']
    password = os.environ['PAYMEE_SONTATA_DB_PASSWORD']
    db = "Sonata_Connect"
    server = "172.17.130.216"
    engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")
    sql_query = f"""SET NOCOUNT ON;EXEC [Sonata_Connect].[dbo].[CSP_STAGE_RESOLUTION] @userid = {rgn_id};SET NOCOUNT OFF"""
    sp_df=pd.read_sql_query(sql_query,engine)
    userid_list = sp_df['UserID'].unique().tolist()
    if user_id in userid_list:
        return True
    else:
        return False

In [88]:
sp_df

,DIVID,DIVNAME,RGNID,RGNNAME,HUBID,HUBNAME,BRANCHID,BRANCHNAME,DisbursemmntID,ACTUALPAIDDATE,POS,OpeningDPD,CusrrentDPD,STAGE_REF,USERID,BRONAME
0,2155,Jaipur,1009,Bharatpur,575,Bharatpur,264,Bharatpur City,6787886,NaT,11027,212,240,3,10803,VIPATI RAM
1,2155,Jaipur,1009,Bharatpur,575,Bharatpur,264,Bharatpur City,6881764,NaT,12282,58,86,2,10804,BABALOO
2,2155,Jaipur,1022,Jaipur,573,Alawar,396,Govindgarh,6737754,NaT,3134,84,112,2,25320,GEETAM SINGH
3,2155,Jaipur,1009,Bharatpur,575,Bharatpur,266,Govardhan,6757459,NaT,11372,185,213,3,29571,CHANDRAKANT
4,2155,Jaipur,1022,Jaipur,2285,Khairthal,475,Tijara,6788317,NaT,13259,212,226,3,29609,SATISH KUMAR SURELA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971,2155,Jaipur,1009,Bharatpur,575,Bharatpur,239,Dholpur,6709451,NaT,5417,141,155,3,32033,ANKIT SINGH
1972,2155,Jaipur,1009,Bharatpur,575,Bharatpur,239,Dholpur,6662470,NaT,500,98,126,3,32033,ANKIT SINGH
1973,2155,Jaipur,1009,Bharatpur,575,Bharatpur,239,Dholpur,7464755,NaT,21433,32,46,2,32033,ANKIT SINGH
1974,2155,Jaipur,1009,Bharatpur,575,Bharatpur,239,Dholpur,7366468,NaT,19480,70,98,2,32033,ANKIT SINGH


In [89]:
reqdf.head()

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
0,2023-04-01,NaN,NaN,NaN,NaN,29938,2271,1,129116,7,Customer (Customer id: Sdl:Bhr:01:006) is inte...,2023-04-01,Completed,2023-04-21,None,None
1,2023-04-01,NaN,NaN,NaN,NaN,30482,27199,1,129117,10,Customer (Customer id: Rab:cit:01:236:037) has...,2023-04-01,Pending,None,None,None
2,2023-04-01,NaN,NaN,NaN,NaN,27245,27199,1,129118,10,Customer (Customer id: Bab:cit:01:189:043) has...,2023-04-01,Pending,None,None,None
3,2023-04-01,NaN,NaN,NaN,NaN,27815,27815,1,129119,3,Customer (Customer id: 1397268) will pay an am...,2023-04-01,Pending,None,None,None
4,2023-04-01,NaN,NaN,NaN,NaN,31063,27199,1,129120,10,Customer (Customer id: Lko:mhn:01:116:004) has...,2023-04-01,Pending,None,None,None


In [90]:
df[df['UserID'] == 9731] 

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName


In [91]:
a = "Customer (Customer id: 1397268) will pay an amount of 0 on 2023-04-01 10:30am by Cash|Collect at Customer's PlaceCustomer name Sunita bai   |contact number 9179450207|"